# Matris Çarpanlarına Ayırma ( Matrix Factorization )

In [1]:
# Matrix factorization bir tablodaki boşlukların bir modele uygun biçimde doldurulmasıdır.

# Örneğin bu serimizin bir önceki bölümünde user-based collaborative filtering ile tavsiyelerde bulunmuştuk;
# Bu tavsiyeleri filmler ve bu filmlere verilen ratingler üzerinden yapmıştık.
# Hatırlamak amacıyla ilgili tablomuzu çağıralım.

import pandas as pd

user_movie_pivot = pd.read_csv('datasets/user_movie_pivot.csv')
user_movie_pivot

,"'burbs, The (1989)",(500) Days of Summer (2009),*batteries not included (1987),...And Justice for All (1979),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),102 Dalmatians (2000),12 Angry Men (1957),...,Zero Dark Thirty (2012),Zero Effect (1998),Zodiac (2007),Zombieland (2009),Zoolander (2001),Zulu (1964),[REC] (2007),eXistenZ (1999),xXx (2002),¡Three Amigos! (1986)
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138488,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138489,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138490,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138491,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Yukarıda görüldüğü üzere tablo üzerinde birçok kullanıcı bir çok filme oy vermemiştir. Dolayısıyla tabloda boş değerler vardır. Boşlukları doldurmak için user'lar ve movie'ler için var olduğu varsayılan Latent Feature'ların (gizli özelliklerin) ağırlıkları var olan veri üzerinden bulunur ve bu ağırlıklar ile var olmayan gözlemler için tahmin yapılır.

Elimizde yukarıdaki gibi bir user-item matrix'i bulunmaktadır.
- User-Item matrix'ini 2 tane daha az boyutlu matrix'e ayrıştıracağız.
- 2 matrix'ten User-Item matrisine gidişin latenet factör'ler (gizli özelliklerle) ile gerçekleştiği varsayımında bulunacağız.
- Dolu olan gözlemler üzerinden latent factor'lerin ağırlıklarını bulacağız.
- Bulunan ağırlıklar ile boş olan gözlemleri dolduracağız.


In [2]:
    #                  Movies (n)
    #              ___________________                             ___         ____________________
    #             |                   |                           |   |    x  |  Movie Factors (q) | k  x n
    #             |                   |                           |   |        ¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯
    #             |                   |                           |   |            r: rating
    # Users (m)   |    Ratings (r)    |      ≈    User Factors (p)|   |            p: usesr factors
    #             |                   |                           |   |            q: movie factors
    #             |                   |                           |   |
    #             |___________________|                           |___| m x k

Biraz daha açıklayıcı olmak gerekirse, elimizde bir tablo var. Bu tablo kullanıcıların spesifik bir filme verdikleri puanları gösteriyor ve bir kullanıcı her filme doğal olarak puan veremez.

Bir kullanıcının bir filmde sevdiği özellikler veya sevmediği özellikler olabilir. Mesela kullanıcı filmin aksiyon filmi olmasını sevebilir veya korku filmi olmasını sevmeyebilir. Başka bir örnek ise kullanıcı için bir filmde Brad Pitt'in olması kullanıcı açısından o filmi  ÇOK sevilebilir kılarken, Brad Pitt'in olmaması o kullanıcı için filmi AZ sevilebilir kılabilir. Biz buna User Factors diyeceğiz. Bunları direk olarak bilemediğimiz için bunlar aynı zamanda latent feature'dır.

Aynı zamanda bir filmin kendine has özellikleri de filmin sevilebilirliğini etkileyebilir. Yani filmin bütçesinin yüksek olması daha kaliteli oyuncuların oynaması daha iyi görsel efektler bir filmi sevilebilir kılarken, bunların olmaması filmin daha kalitesiz olacağı genel geçer kanısı ile bilmi daha az sevilebilir kılabilir. Bir başka örnek olarak ise filmin yönetmeni verilebilir. Bir filmin yönetmeninin bakış açısı filmin sevilebilirliğini etkileyebilir. Biz bunlara da Movie Factors diyeceğiz. Bunları da direk olarak bilemediğimiz için bunlar aynı zamanda latent feature'dır.

In [3]:
# Konuya bir de şu şekilde yaklaşacağız,

# yukarıdaki User Factors matrisinin alt kısmına baktığımızda m x k ifadesini görüyoruz,
# m tane user var, k tane factor var diyebiliriz.

# yukarıdaki Movie Factors matrisinin alt kısmına baktığımızda k x n ifadesini görüyoruz,
# n tane film var, k tane factor var diyebiliriz.

In [4]:
# Ayrıca şöyle de diyebiliriz, bir kullanıcı bir filmi seviyorsa o kullanıcının filmi sevmesi için bazı faktörler vardır.
# Bu sebepler şu anda ölçülebilir değildir, lakin kullanıcının vermiş olduğuğ puanı oluşturan gizli faktörlerdir diyebiliriz.

# Az önceki faktörlerin filmlerde de karşılığı vardır. Filmlerin içerisinde de filmin beğenilmesini sağlayan faktörler vardır.

- Rating matrisinin iki factor matrisinin (dot product) ile oluştuğu varsayılır.

- Factor matrisleri ise user latent factors ve movie latent factors olarak ikiye ayırabiliriz.

- Latent factors karşımıza; latent features, gizli faktörler veya gizli değişkenler olarak gelebilir.

- Kullanıcıların ve filmlerin latent feature'lar için skorlara sahip olduğu düşünülür.

- Bu ağırlıklar (skorlar) önce var olan veri üzerinden bulunut ve sonra BLANK bölümler bu ağırlıklara göre doldurulur.

In [5]:
# Bu faktörler filmler için; komedi, korku, macera vs. olup olmaması,
#                            veya belirli bir oyuncunun, yönetmenin vs olup olmaması olabilir.

# Bu movie faktörlerinin kullanıcıdaki karşılıkları da user factors olacaktır.


<p align="center" >    
    <img src = "grafikler/matrixfactorization.png" />
</p>

In [6]:
# U1 kullanıcının, M1 filmine verdiği rating'i r11 diye isimlendirelim;

# Teoriye göre;

# r11 = p11 x q11 + p12 x q21 eşitliği ile belirlenir. 

# Aynı şekilde r21 ise;

# r21 = p21 x q11 + p22 x q21  eşitliği ile belirlenir.

Şimdi buraya kadar işlemimizin teorisini anlatmaya çalıştık.

Buradan sonra ise;

- var olan değerler üzerinden iteratif bir şekilde tüm p ve q'lar bulunur ve sonra kullanılır.

                Burada bizi bir sorun beklemektedir. p ve q'lar nasıl bulunacaktır.
- Başlangıçta rasgele p ve q değerleri iler rating matrisindeki değerler tahmin edilmeye çalışılır. Sonrasında aradaki farkın karelerinin en küçük olacak şekilde optimize edilmeye çalışılır.

In [7]:
# Yukarıda p ve q değerlerine rasgele değer vereceğimizi daha sonrasında ise bu değerler ile boş ratingleri tahmin edeceğimizi ifade etmiştik.
# Bu tahminler ile gerçek değerler arasındaki farkın kareleri toplamı ise "Mean Square Error" ile ifade edilir.
# Negatif değerler ile pozitif değerlerin birbirini götürmemesi için farkların kareleri alınmaktadır.
# Kare işlemi olduğu için daha sonra MSE'nin karekökü alınarak "Root Mean Square Error" hesaplanabilir.

# Bizler MSE'nin veya RMSE'nin minimum olmasını isteriz.
# Bunun için ise p ve q'nun en optimize değerler olması gerekir ki RMSE en küçük olsun.

In [8]:
# Burada yöntem olarak Gradient Descent kullanacağız.
# Gradient Descent fonksiyon minimizasyonu için kullanılan bir optimizasyon yöntemidir.

Gradient Descent ile

Gradyanın negatidi olarak tanımlanan "en dik iniş" yönünde iterarif olarak parametre değerini güncelleyerek ilgili fonksiyonun minimum değerini vereek parametreleri buluruz.

In [9]:
# Yukarıdaki cümle bir hayli karmaşık gibi gözükmektedir ve bunu parça parça ele almak zorundayız.

# Lakin şunu ifade etmemiz gerekmektedir, diyelim ki bizim bir rating'imiz 5 olsun ve biz bu rating için 1 değerinde bir tahminde bulunalım.
# Şimdi bizim tahminimizin hatalı olduğu açıkça gözükmektedir, çünkü (5-1)^2 = 16'dır. Bizim bu hatayı azalmamız gerekmektedir.
# Bir sonraki tahmin değerimiz olarak rating'imiz 2 olsun. (5-2)^2 = 9'dur. Hatamızı azalttık.
# Bu işlemi MSE'miz minimum olana kadar bütün user'lar ve movie'ler için yapacağız.


# Yani "Gradient Descent" ile türeve dayalı olarak ağırlık matrisindeki p ve q işlemlerini iteratif olarak değiştireceğiz.

# Şimdi yukarıdaki tanımımızdan hareket edersek;

# Gradyan eğim ve türev demektir. Zaten hatırlarsak bir fonksiyonun belirli bir noktadaki türevi o noktadaki eğimini bize verirdi.
# Dolayısıyla bir fonksiyonun belirli bir noktadaki türevi, o noktadaki maksimum artış hızını verir.

# Dolayısıyla gradyanının (eğiminin veya türevinin) negatifine doğru gittiğimizde ve bu yöne iteratif olarak (her bir kullanıcı ve film için)
# ilgili fonksiyonu minimumlaştıracak şekilde parametreler bulunur.

In [10]:
# Artık kütüphanelerimizi iport edip sonrasında verilerimizi işleme vakti geldi :)

import pandas as pd
from surprise import Reader, SVD, Dataset, accuracy
from surprise.model_selection import GridSearchCV, train_test_split, cross_validate

In [11]:
# Adım 1: Veri Setinin Hazırlanması
# Adım 2: Modelleme
# Adım 3: Model Tuning
# Adım 4: Final Model ve Tahmin

### Adım 1: Veri Setinin Hazırlanması

In [12]:
# Burada bütün veri seti üzerinden değil bir kaç film seçerek bu uygulamayı yürütüyor olacağız.

movie = pd.read_csv('datasets/movie_lens_dataset/movie.csv')
rating = pd.read_csv('datasets/movie_lens_dataset/rating.csv')
df = movie.merge(rating, how="left", on="movieId")
df.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.0,4.0,1999-12-11 13:36:47
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,6.0,5.0,1997-03-13 17:50:52
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8.0,4.0,1996-06-05 13:37:51
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10.0,4.0,1999-11-25 02:44:47
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11.0,4.5,2009-01-02 01:13:41


In [13]:
# Takip edilebilirlik açısından 4 film seçeceğiz.

movie_ids = [130219, 356, 4422, 541]
movies = ["The Dark Knight (2011)",
          "Cries and Whispers (Viskningar och rop) (1972)",
          "Forrest Gump (1994)",
          "Blade Runner (1982)"]

In [14]:
sample_df = df[df.movieId.isin(movie_ids)]
sample_df.head()

,movieId,title,genres,userId,rating,timestamp
2457839,356,Forrest Gump (1994),Comedy|Drama|Romance|War,4.0,4.0,1996-08-24 09:28:42
2457840,356,Forrest Gump (1994),Comedy|Drama|Romance|War,7.0,4.0,2002-01-16 19:02:55
2457841,356,Forrest Gump (1994),Comedy|Drama|Romance|War,8.0,5.0,1996-06-05 13:44:19
2457842,356,Forrest Gump (1994),Comedy|Drama|Romance|War,9.0,4.0,2001-07-01 20:26:38
2457843,356,Forrest Gump (1994),Comedy|Drama|Romance|War,10.0,3.0,1999-11-25 02:32:02


In [15]:
sample_df.shape

# Seçtiğimiz 4 adet film için 97343 puan bulunmaktadır.

(97343, 6)

In [16]:
# Şimdi satırlara kullanıcıları, sütunlara fimleri ve tablonun içerisine de kullanıcıların filmlere verdikleri puanları
# görebileceğimiz bir pivot tablosunu oluşturacağız.

user_movie_df = sample_df.pivot_table(index=["userId"],
                                      columns=["title"],
                                      values="rating")

user_movie_df

title,Blade Runner (1982),Cries and Whispers (Viskningar och rop) (1972),Forrest Gump (1994),The Dark Knight (2011)
userId,,,,
1.0,4.0,NaN,NaN,NaN
2.0,5.0,NaN,NaN,NaN
3.0,5.0,NaN,NaN,NaN
4.0,NaN,NaN,4.0,NaN
7.0,NaN,NaN,4.0,NaN
...,...,...,...,...
138474.0,NaN,NaN,5.0,NaN
138483.0,4.0,NaN,4.0,NaN
138484.0,NaN,NaN,5.0,NaN


In [17]:
user_movie_df.shape

# 76918 satır kullanıcıları ifade etmektedir.

(76918, 4)

In [18]:
# Reader fonksiyonunu suprise kütüphanesi ile import etmiştik.
# Reader fonksiyonu ile biz skalamızı ifade ediyoruz.

reader = Reader(rating_scale=(1, 5))

In [19]:
# Yine suprise kütüphanesi bizden veriyi belirli bir formatta ister 
# ve bunun için de "Dataset" fonksiyonu ve ".load_from_df()" metodunu kullanırız.
# ".load_from_df()" metodunun içerisine ise satır, sütun, kesişim ve skala bilgilerini göndeririz.

# Kendi documentation'ına da göz atmanın faydalı olduğunu düşünüyorum.
# The dataframe containing the ratings. 
# It must have three columns, corresponding to the user (raw) ids, the item (raw) ids, and the ratings, in this order.

data = Dataset.load_from_df(sample_df[['userId',
                                       'movieId',
                                       'rating']], reader)

# Yukarıdaki kodu çalıştırarak verimizi surprise kütüphanesinin istediği formaya getirmiş olduk.

### Adım 2: Modelleme

In [20]:
# Makine öğrenmesine daha giriş yapmadık lakin; bu şekildeki öğrenmelerde (makine öğrenmesi veya istatistiksel öğrenme vs. de olabilir)
# Veri setimizi train ve test olmak üzere ikiye böleriz.

# Bu şekilde modelizimi train'de geliştirir ve verinin test kısmında ise geliştirdiğimiz modeli deneriz (test ederiz).

# Bu şekilde verimizin öğrendiği veride gözden kaçırdığımız bir şeyden dolayı çok iyi sonuç verip, 
# diğer verilerde hatalı sonuçlar vermesinin önüne geçmeye çalışırız.

In [21]:
# Veriyi bulmak için scikit-learn değil, suprise içerisindeki "train_test_split" fonksiyonunu çağıracağız.

trainset, testset = train_test_split(data, test_size=.25)

In [22]:
# Bir sonraki aşamada ise model nesnemizi oluşturacağız.

svd_model = SVD()

# Peki nedir bu model nesnesi, matrix factorization yöntemini kullanacak olduğumuz fonksiyon olacaktır.

In [23]:
# Bir önceki kısımda nesnemizi oluşturduk ve bu nesnemizi kullanacarak, 
# train dataset (trainset) ile beraber bir model kurma (fit etme) işlemi yapacağız.

svd_model.fit(trainset)

In [24]:
# Modelimizi fit ettik ve şimdi test dataseti (testset) üzerinde bir test edelim;

predictions = svd_model.test(testset)
predictions

[Prediction(uid=127014.0, iid=356, r_ui=4.0, est=4.0370562588277865, details={'was_impossible': False}),
 Prediction(uid=110990.0, iid=541, r_ui=5.0, est=4.18133124043154, details={'was_impossible': False}),
 Prediction(uid=36440.0, iid=541, r_ui=5.0, est=4.056819754437834, details={'was_impossible': False}),
 Prediction(uid=18119.0, iid=356, r_ui=4.5, est=4.0370562588277865, details={'was_impossible': False}),
 Prediction(uid=38006.0, iid=541, r_ui=3.5, est=4.18133124043154, details={'was_impossible': False}),
 Prediction(uid=102216.0, iid=356, r_ui=0.5, est=4.0370562588277865, details={'was_impossible': False}),
 Prediction(uid=77932.0, iid=356, r_ui=5.0, est=4.0370562588277865, details={'was_impossible': False}),
 Prediction(uid=16424.0, iid=541, r_ui=4.0, est=4.048103753615669, details={'was_impossible': False}),
 Prediction(uid=104816.0, iid=356, r_ui=5.0, est=4.0370562588277865, details={'was_impossible': False}),
 Prediction(uid=81254.0, iid=541, r_ui=4.0, est=4.18133124043154, 

In [25]:
# Şimdi çıktıyı bir yorumlamaya çalışalım;

# Prediction(uid=3685.0, iid=356, r_ui=4.5, est=4.090432247214464, details={'was_impossible': False})

# uid=3685.0 ID'li kullanıcının, id=356 ID'li filme verdiği
# r_ui=4.5 gerçek puan iken biz bu puanı est=4.090432247214464 olarak tahmin etmişiz.

Yukarıda görüldüğü üzere gerçek değerlerim ile tahmin değerlerim arasında farklılılar bulunmaktadır.

Bizim bu farklılıkları görebilmemiz gerekir.

In [26]:
# Bu farklılılaı görebilmek için ise accuract nesnesini ve ".rmse()" metodunu kullanırız.

accuracy.rmse(predictions)

# Bu ne demektir;

# Bir kullanıcı için beklenen ortalama hatadır.

RMSE: 0.9326


0.9325846934203301

In [27]:
# Spesifik bir kullanıcı ve film için bir tahminde bulunmak istersek;

svd_model.predict(uid=1.0, iid=541, verbose=True)

# ID no'su 1 olan kullanıcının ID no'su 541 olan filme (ki film "Blade Runner (1982)") verdiği puanı tahmin etmeye çalışacağız.

user: 1.0        item: 541        r_ui = None   est = 4.18   {'was_impossible': False}


Prediction(uid=1.0, iid=541, r_ui=None, est=4.18133124043154, details={'was_impossible': False})

In [28]:
# Bir de gerçek değerine bakalım;

sample_df[sample_df["userId"] == 1]

# Biz bu değeri 4.07 olarak tahmin ederken filme verilen gerçek puan ise 4'ttür.
# Modeli her eğittiğimizde bu değerler değişiklik gösterebilir.
# Zira modelin train ve test olarak nerelerden bölündüğünü tam olarak bilemiyoruz.

,movieId,title,genres,userId,rating,timestamp
3612352,541,Blade Runner (1982),Action|Sci-Fi|Thriller,1.0,4.0,2005-04-02 23:30:03


### Adım 3: Model Tuning (Model Kurma)

In [29]:
# Burada modelimizi optimize etmeye çalışacağız.
# Modeli optimize etmek modeli tahmin performansını arttırmaya çalışmak demektir.

# Bir modeli optimize etmeninin bir çok yolu olmasına rağmen;
# Biz burada modelin dışsal parametrelerini (yani kullanıcı tarafından ayarlanabilen veya hiper parametrelerini) "Nasıl optimize ederiz?"
# sorusuna cevap arıyor olacağız.

# Bizim burada müdahale edeceğimiz parametrelerden bir tanesi iteraasyon (tekrar) sayısıdır.
# Yukarıdaki tanımdan hatırlarsak p ve q parametrelerimizi iteratif olarak değiştiriyorduk.
# Yani biz kaç defa bu ağırlıkları güncelleyeceğiz.

# Bu parametrenin kullanıcı tarafından verilmesi veya optimize edilmesi gerekir.

# SVD() nesnesinin içerine bakıldığında, 
# içerisindeki parametrelerden bir tanesi
#   n_factors parametresi bulunmaktadır ve default'u son versiyonda 20 olarak gözükmektedir.
#   faktör sayısı arttılarak veya azaltılarak RMSE sayısı kontrol edilirse n_factors'ün arttırılıp azaltılmasına ilişkin yorum yapılabilir.
# SVD() içerisindeki bir başka parametre ise;
#   n_epochs parametresidir. Bu parametre ise iterasyon sayısını ifade eder. Ön tanımlı değeri ise 20'dir.

# Bunlara bizler daşsal olarak müdahale edebiliriz.

In [30]:
# Bu dışsal değerleri modelimize gönderebilmek için bir parametre ızgarası (parameter_grid) hazırlayacağız.

param_grid = {'n_epochs': [5, 10, 20],
              'lr_all': [0.002, 0.005, 0.007]}

# Yukarıdaki sözlükte ifade edilen şey; "n_epochs" değeri 5 iken VE "lr_all" değeri 0.02 iken modeli p ve q değerlerini hesapla diyoruz.

In [31]:
# Aşağıda ise verilen parametreleri farklı farklı çalıştıracak ve modeli arayacak bir fonksiyona ihtiyacımız olacak;

gs = GridSearchCV(SVD,
                  param_grid,
                  measures=['rmse', 'mae'],
                  cv=3,
                  n_jobs=-1,
                  joblib_verbose=True)

# Yukarıda "GridSearchCV()" fonksiyonunun içerisine;
# SVD nesnesini gönderiyoruz,
# parametre ızgarası gönderiyoruz,
# hatamızı hangi değerlere göre değerlendireceğimizi söylüyoruz (yukarıda RMSE ve MAE'a (mean absolute error) bakacağımızı ifade ettik),
# "cv=" parametresi ise bize çapraz sorgulama yapmak isteyip istemediğimizi sorar,
#   "cv=" parametresine girdiğimiz 3 değeri ise veri setini 3'e bölüp 2 set ile model kurup 1 set ile test edeceğini,
#   daha sonrasında diğer iki set ile model kurup farklı 1 parça ile test edeceğini
#   en sonunda ise kalan test edilen 2 parça ile modelin kurulup son parça ile modelin test edileceğini ifade eder.
#   En sonunda ise bu test işlemlerinin ortalamasını alır.
# "n_jobs=-1" işlemcileri full performans ile kullanması gerektiğini ifade eden parametredir.
# "joblib_verbose=True" ile de işlemler gerçekleşirken raporlama istediğimi ifade ediyorum.

In [32]:
gs.fit(data)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:    6.4s finished


In [33]:
# Şimdi ise gs içerisindeki en iyi RMSE'ye sahip olan modelin RMSE'sini çağıralım,

gs.best_score['rmse']

0.9305790433768855

In [34]:
# Bu sonucu veren en iyi parametreleri çağırmak istersek;

gs.best_params['rmse']

{'n_epochs': 10, 'lr_all': 0.002}

### Adım 4: Final Model ve Tahmin

In [35]:
# SVD modelin çağırılabilecek metodlarına bir göz atalım.

dir(svd_model)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slotnames__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'bi',
 'biased',
 'bsl_options',
 'bu',
 'compute_baselines',
 'compute_similarities',
 'default_prediction',
 'estimate',
 'fit',
 'get_neighbors',
 'init_mean',
 'init_std_dev',
 'lr_bi',
 'lr_bu',
 'lr_pu',
 'lr_qi',
 'n_epochs',
 'n_factors',
 'predict',
 'pu',
 'qi',
 'random_state',
 'reg_bi',
 'reg_bu',
 'reg_pu',
 'reg_qi',
 'sgd',
 'sim_options',
 'test',
 'trainset',
 'verbose']

In [36]:
svd_model.n_epochs

20

In [37]:
# Yukarıdaki modelimizde baktığımızda, en iyi sonucu veren parametrelerin, 
# SVD() fonksiyonundaki ön tanımlı değerlerinden farklı olabileceğini fark ettik.

# Bu sebeple bizim bu parametrelerle yeni bir model kurmamız gerekir.

svd_model = SVD(**gs.best_params['rmse'])

# Modelizimi yukarıdaki gibi kurabiliriz. 
# Dilersek kwargs (key-worded arguman) yerine elimizle teker teker {'n_epochs': 5, 'lr_all': 0.002} şeklinde de girebiliriz.

In [38]:
# Normalde biz verimizi train ve test diye ikiye bölmüştük,

# Biz yukarıda hiper parametrelerimizi optimize ettik ve hatalarımıza baktık, ne noktada olduğumuzu öğrendik,
# Bu sebeple modelleme işleminde verimizi train ve test diye ayırmamıza gerek kalmadan bütün veriyi gösterme yoluna gidebilirim.

data = data.build_full_trainset()

In [39]:
# Artık modelimizi fit etme vakti geldi :)

svd_model.fit(data)

In [40]:
# Bir de bir kıyaslama yapalım;

svd_model.predict(uid=1.0, iid=541, verbose=True)

user: 1.0        item: 541        r_ui = None   est = 4.24   {'was_impossible': False}


Prediction(uid=1.0, iid=541, r_ui=None, est=4.237804199565605, details={'was_impossible': False})

In [41]:
sample_df[sample_df["userId"] == 1]

,movieId,title,genres,userId,rating,timestamp
3612352,541,Blade Runner (1982),Action|Sci-Fi|Thriller,1.0,4.0,2005-04-02 23:30:03


In [42]:
# Modelimiz 4.0 olarak verilen rating'i 4.18 olarak tahmin etmiş.